# 01 数据集的导入和基本信息
train数据集中包括的特征有：
PassengerId：乘客的编号
Survived：是否存活
Pclass：票的类型，1是最好的、3是最差的
Name:性命
Sex:年龄
Age:年龄
SibSp:相伴的兄弟姐妹以及配偶
Parch：相伴的父母或者孩子
Ticker：乘客的票号
Fare:乘客的票价
Cabin：机舱号码
Embarked：下车地点

In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [75]:
train_data=pd.read_csv('../titanic/train.csv')
test_data=pd.read_csv('../titanic/test.csv')

In [76]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [77]:
# 和交通数据中一样拿到数据需要考虑数据是否存在缺失、异常、或者是重复等等
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [78]:
# 可以看出数据中存在缺失的特征包括：Age、Cabin、Embarked

# 02 数据集的预处理

In [99]:
## 缺失值的修补
#Embarked 上船地点中缺少两个数，对于之后并没有太大的影响，可以使用众数进行修补
train_data.Embarked.fillna(value=train_data.Embarked.dropna().mode().values[0])
data_test=test_data.Fare.fillna(value=test_data.Fare.dropna().mean())
test_data.loc[:,'Fare']=data_test
# 从直觉上来看，年龄对于之后是否坠船有相关的影响，因此在这里选择对应的非空特征进行绚练
feature_name=['Pclass','Sex', 'SibSp', 'Parch','Fare','Embarked','Survived']
# cabin 的机场缺失太多，这里就不作为训练来

In [86]:
# 对性别进行编码
train_data_copy=train_data[feature_name]
from sklearn.preprocessing import LabelEncoder

In [87]:
label=LabelEncoder()

train_data_copy.loc[:,'Sex']=label.fit_transform(train_data_copy.Sex)
train_data_copy.loc[:,'Embarked']=label.fit_transform(train_data_copy.Embarked)

test_data.loc[:,'Sex']=label.fit_transform(test_data.Sex)
test_data.loc[:,'Embarked']=label.fit_transform(test_data.Embarked)

/var/folders/6s/d3zr9my94njd1q1hrj7t4bmr0000gn/T/ipykernel_75812/1376438959.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_copy.loc[:,'Sex']=label.fit_transform(train_data_copy.Sex)
/var/folders/6s/d3zr9my94njd1q1hrj7t4bmr0000gn/T/ipykernel_75812/1376438959.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_copy.loc[:,'Embarked']=label.fit_transform(train_data_copy.Embarked)


In [88]:
## 尝试使用SVM
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import LinearSVC

p_s_c=Pipeline([
    ('poly_features',PolynomialFeatures(degree=3)),
    ('scaler',StandardScaler()),
    ('svm_clf',LinearSVC(C=10,loss='hinge'))
])
p_s_c.fit(train_data_copy.loc[:,('Pclass','Sex', 'SibSp', 'Parch','Fare','Embarked')],train_data_copy.iloc[:,-1])

/Users/chenxia/miniforge3/envs/tf2/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf', LinearSVC(C=10, loss='hinge'))])

In [103]:
from sklearn.metrics import accuracy_score
result=p_s_c.predict(test_data.loc[:,('Pclass','Sex', 'SibSp', 'Parch','Fare','Embarked')])

In [105]:
submisstion=pd.DataFrame({
    'PassengerId':test_data['PassengerId'],
    'Survived':result
})
submisstion.to_csv('submission.csv',index=False)